In [1]:
from torchsummary import summary
import segmentation_models_pytorch as smp
import os
import torch
from torchvision import transforms as T
from PIL import Image
import numpy as np

import cv2
import pandas as pd

In [2]:
os.listdir()

['.ipynb_checkpoints',
 '1704786132771_d9b4215b-cdf5-4e21-96c8-e05edc7cfab9.mp4',
 '1711791982129_b655f649-7487-4c5b-8bc6-b82b42773351.mp4',
 '1714125130126_0847b0cb-33a9-4905-a1b1-5b7b51201061.mp4',
 '1714125130126_0847b0cb-33a9-4905-a1b1-5b7b51201061cccc.mp4',
 '1717143752238_2660e884-da2d-42ab-a444-100597d25450.mp4',
 'deeplab-efficientnet-b7.pt',
 'deeplabv3-state_sm_effidentv.pt',
 'images.csv',
 'snapshot.ipynb',
 'snapshots35',
 'WhatsApp Video 2024-10-18 at 10.00.07.mp4',
 'WhatsApp Video 2024-10-18 at rotated.mp4',
 'WhatsApp Video rotated.mp4',
 'WhatsAppcccc.mp4']

In [3]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
model_path = 'deeplab-efficientnet-b7.pt'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = smp.DeepLabV3Plus('efficientnet-b7', classes=42, encoder_depth=5)
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
model.to(device)
model.eval()

DeepLabV3Plus(
  (encoder): EfficientNetEncoder(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 64, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d((0, 1, 0, 1))
    )
    (_bn0): BatchNorm2d(64, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          64, 64, kernel_size=(3, 3), stride=[1, 1], groups=64, bias=False
          (static_padding): ZeroPad2d((1, 1, 1, 1))
        )
        (_bn1): BatchNorm2d(64, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          64, 16, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          16, 64, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_project_conv): Conv2dStatic

In [4]:
CLASSES_CARPARTS =  {
    0:'bg',
 1: 'Antenna',
 2: 'FogLight',
 3: 'FrontBumper',
 4: 'FrontLid',
 5: 'FrontLight',
 6: 'FrontWindShield',
 7: 'FuelLid',
 8: 'Grille',
 9: 'Jack',
 10: 'LeftBackDoor',
 11: 'Rear Left Wheel', #Rear Left Wheel
 12: 'LeftFrontDoor',
 13: 'LeftFrontFender',
 14: 'LeftFrontSills',
 15: 'Front Left Wheel', #LeftFrontWheel
 16: 'LeftRearFender',
 17: 'LeftRearSills',
 18: 'LeftSideMirror',
 19: 'LeftSideSills',
 20: 'OpenTrunkLid',
 21: 'RearBumper',
 22: 'RearWindShield',
 23: 'RightBackDoor',
 24: 'Rear Right Wheel', #Rear Right Wheel
 25: 'RightFrontDoor',
 26: 'RightFrontFender',
 27: 'RightFrontSills',
 28: 'Front Right Wheel', #Front Right Wheel
 29: 'RightRearFender',
 30: 'RightRearSills',
 31: 'RightSideFender',
 32: 'RightSideMirror',
 33: 'RightSideSills',
 34: 'Silencer',
 35: 'SpareWheel',
 36: 'Sunroof',
 37: 'TailLight',
 38: 'Top',
 39: 'TrunkLid',
 40: 'Wipers'}

In [5]:
CLASSES_CARPARTS_inverted = {value:key for key,value in CLASSES_CARPARTS.items()}

In [6]:
# def predict(image, class_list):
#     print(image)
#     image = Image.open(image)
#     pred_mask, output = predict_image_mask_miou(image)

#     flat_mask = pred_mask.flatten()
#     flat_mask = flat_mask.numpy()
#     flat_mask = np.delete(flat_mask,0)
# #     print(flat_mask.numpy())
#     unique_classes = np.unique(flat_mask)
# #     unique_classes = flat_mask.unique()
#     unique_classes = unique_classes[unique_classes != 0]
#     class_percentages = {}
#     for u in unique_classes:
#         class_percentage = round((flat_mask == u).mean().item() * 100,2)
#         if class_percentage >= 1 or int(u) in ([17, 10, 16, 14, 15, 6]):
#             class_percentages[CLASSES_CARPARTS[int(u)]] = class_percentage
#     return class_percentages, pred_mask, image

In [7]:
def predict(image, class_list):
    print(image)
    image = Image.open(image)
    pred_mask, output = predict_image_mask_miou(image)

    flat_mask = pred_mask.flatten()
    flat_mask = flat_mask.numpy()
    flat_mask = np.delete(flat_mask,0)
#     print(flat_mask.numpy())
    unique_classes = np.unique(flat_mask)
#     unique_classes = flat_mask.unique()
    unique_classes = unique_classes[unique_classes != 0]
    class_percentages = {}
    for u in unique_classes:
        class_percentage = (flat_mask == u).sum().item()
        if class_percentage >= 1 or int(u) in ([17, 10, 16, 14, 15, 6]):
            class_percentages[CLASSES_CARPARTS[int(u)]] = class_percentage
    return class_percentages, pred_mask, image

In [8]:
def predict_image_mask_miou(image, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
        t = T.Compose([T.Resize((1280, 1088 )), T.ToTensor(), T.Normalize(mean, std)])
        image = t(image)
#         image = image.to(self.device)
        with torch.no_grad():        
            image = image.unsqueeze(0)
            output = model(image)
            masked = torch.argmax(output, dim=1)
            masked = masked.cpu().squeeze(0)
        return masked, output

In [53]:

# from brisque import BRISQUE

def save_non_repeating_snapshots(video_path, threshold=0.9, output_folder='snapshots37', frame_interval=10):
    # Load the video
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video file")
        return

    # Read the first frame
    ret, prev_frame = cap.read()
    if not ret:
        print("Error reading first frame")
        return

    frame_index = 0
    snapshot_index = 0
    flag = 1
    
    parts_lst = []
    
    while True:
        # Read next frame
        ret, frame = cap.read()
        if not ret:
            break
        
        grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        laplacian_score = cv2.Laplacian(grey, cv2.CV_64F).var()
#         print(laplacian_score)
        if laplacian_score > 80:
            
            if frame_index % frame_interval == 0:

                diff = cv2.absdiff(prev_frame, frame)
                non_zero_count = cv2.countNonZero(cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY))

                # Check if the change between frames is significant
                if non_zero_count / (diff.size / 3) > threshold:
                  
                    snapshot_path = f"{output_folder}/9999snapshot_{snapshot_index}.jpg"
                    snapshot_index = snapshot_index + 1
                    cv2.imwrite(snapshot_path, frame)
                    
                    
                    class_percentages, mask, image = predict(snapshot_path, CLASSES_CARPARTS)
#                     cv2.imwrite(snapshot_path, image) 
                    part_coordinate = {}
                    for cls in class_percentages:
                        
                        target_mask = np.where(mask == CLASSES_CARPARTS_inverted[cls], 255, 0).astype(np.uint8)
                        
                        if cls not in ['BackDoor', 'FrontDoor', 'Sidemirror']:
                            kernel_size = 15  # You can adjust this size depending on how much you want to dilate
                            kernel = np.ones((kernel_size, kernel_size), np.uint8)
                            target_mask = cv2.dilate(target_mask, kernel, iterations=1)
                        
                        contours, _ = cv2.findContours(target_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                
                        largest_contour = max(contours, key=cv2.contourArea)
                        x, y, w, h = cv2.boundingRect(largest_contour)
                        
                        padding = 50  # Adjust padding as needed
                        x = max(x - padding, 0)
                        y = max(y - padding, 0)
                        w = min(w + 2 * padding, image.width - x)
                        h = min(h + 2 * padding, image.height - y)
            
                        part_coordinate['coordinates_'+cls] = [x, y, x + w, y + h]

                    class_percentages['pred_cordinates'] = part_coordinate
                    class_percentages['image_name'] = snapshot_path
                    parts_lst.append(class_percentages)

                    prev_frame = frame  # Update the previous frame to the current one for comparison
                    flag = 1
                    
                    frame_index = frame_index + frame_interval

            frame_index += 1
            # break
            # if frame_index > 100:
            #     break
    
    
    cap.release()
    return parts_lst 
    

# Usage1703239183845_f83454b1-0032-4f37-9346-56cf61eb3b2c
# parts_lst = save_non_repeating_snapshots('1703239183845_f83454b1-0032-4f37-9346-56cf61eb3b2c.mp4')

# parts_lst = save_non_repeating_snapshots('1704786132771_d9b4215b-cdf5-4e21-96c8-e05edc7cfab9.mp4')

parts_lst = save_non_repeating_snapshots('WhatsApp Video 2025-01-24 at 07.50.43.mp4')

# save_non_repeating_snapshots('1677131037223_55182577-2e1c-4593-901a-8498f6f1d8ae.mp4')
# save_non_repeating_snapshots('1683263317275_676b75ea-fe29-4af5-9e7e-0358dec57457.mp4')

snapshots37/9999snapshot_0.jpg
snapshots37/9999snapshot_1.jpg
snapshots37/9999snapshot_2.jpg
snapshots37/9999snapshot_3.jpg
snapshots37/9999snapshot_4.jpg
snapshots37/9999snapshot_5.jpg
snapshots37/9999snapshot_6.jpg
snapshots37/9999snapshot_7.jpg
snapshots37/9999snapshot_8.jpg
snapshots37/9999snapshot_9.jpg
snapshots37/9999snapshot_10.jpg
snapshots37/9999snapshot_11.jpg
snapshots37/9999snapshot_12.jpg
snapshots37/9999snapshot_13.jpg
snapshots37/9999snapshot_14.jpg
snapshots37/9999snapshot_15.jpg
snapshots37/9999snapshot_16.jpg
snapshots37/9999snapshot_17.jpg
snapshots37/9999snapshot_18.jpg
snapshots37/9999snapshot_19.jpg
snapshots37/9999snapshot_20.jpg
snapshots37/9999snapshot_21.jpg
snapshots37/9999snapshot_22.jpg
snapshots37/9999snapshot_23.jpg
snapshots37/9999snapshot_24.jpg
snapshots37/9999snapshot_25.jpg
snapshots37/9999snapshot_26.jpg
snapshots37/9999snapshot_27.jpg
snapshots37/9999snapshot_28.jpg
snapshots37/9999snapshot_29.jpg
snapshots37/9999snapshot_30.jpg
snapshots37/9999sn

In [54]:
dataframe = pd.DataFrame(parts_lst)


In [55]:
dataframe['image_name'].str.split('/',expand=True).iloc[:,-1].str.split('_',expand=True).iloc[:,-1].str.split('.',expand=True).iloc[:,0]

0        0
1        1
2        2
3        3
4        4
      ... 
145    145
146    146
147    147
148    148
149    149
Name: 0, Length: 150, dtype: object

In [59]:
dataframe['FrontMax'] = dataframe['FrontBumper'] + dataframe['FrontLid'] + dataframe['FrontLight'] + dataframe['FrontWindShield']
frontimg = dataframe[dataframe['FrontMax'] == dataframe['FrontMax'].max()]['image_name'].values[0]

In [60]:
# dataframe.to_csv('images.csv',index=False)
frontimg

'snapshots37/9999snapshot_65.jpg'

In [61]:
'FrontBumper', 'FrontLid', 'FrontLight', 'FrontWindShield', 'Grille',
       'RightSideFender', 'TailLight', 'TrunkLid', 'pred_cordinates',
       'image_name', 'FogLight', 'LeftSideMirror', 'Sunroof',
       'RightFrontFender', 'Antenna', 'RightBackDoor', 'RightFrontDoor',
       'LeftFrontFender', 'Rear Right Wheel', 'Front Right Wheel',
       'RightRearSills', 'RearWindShield', 'RightFrontSills',
       'RightRearFender', 'RightSideMirror', 'Silencer', 'RearBumper',
       'LeftFrontDoor', 'Top', 'LeftSideSills', 'Jack', 'LeftBackDoor',
       'Rear Left Wheel', 'LeftRearFender', 'LeftRearSills', 'OpenTrunkLid',
       'Front Left Wheel', 'SpareWheel', 'LeftFrontSills', 'RightSideSills',
       'FrontMax'

IndentationError: unexpected indent (1755945117.py, line 2)

In [81]:
dataframe['BackMax'] = dataframe['TrunkLid'] + dataframe['TailLight'] + dataframe['RearWindShield'] 
backimg_df = dataframe[dataframe['BackMax'] == dataframe['BackMax'].max()].head(1)
backimg = backimg_df['image_name'].values[0]
backimg

'snapshots37/9999snapshot_147.jpg'

In [82]:
dataframe[dataframe.index == 147].T

,147
OpenTrunkLid,NaN
RearBumper,20637.0
RearWindShield,6558.0
Sunroof,12516.0
Top,28949.0
TrunkLid,282.0
pred_cordinates,"{'coordinates_FrontBumper': [493, 969, 480, 36..."
image_name,snapshots37/9999snapshot_147.jpg
Jack,NaN
FrontWindShield,NaN


In [63]:
dataframe['Rightsidemax'] = dataframe['RightRearFender'] + dataframe['RightFrontFender'] + dataframe['TailLight'] + dataframe['FrontLight']
dataframe['Leftsidemax'] = dataframe['LeftRearFender'] + dataframe['LeftFrontFender'] + dataframe['TailLight'] + dataframe['FrontLight']

# firstside = dataframe[dataframe.index < backimg_df.index[0]]
# secondside = dataframe[dataframe.index > backimg_df.index[0]]
# print(firstside['sidemax'].max())


# if not firstside['sidemax'].max():
#     dataframe['sidemax'] = dataframe['RearFender'] + dataframe['FrontFender']
#     firstside = dataframe[dataframe.index < backimg_df.index[0]]
#     secondside = dataframe[dataframe.index > backimg_df.index[0]]
#     print('sdfsdfadf')

# if not firstside['sidemax'].max():
#     dataframe['sidemax'] = dataframe['RearFender']
#     firstside = dataframe[dataframe.index < backimg_df.index[0]]
#     secondside = dataframe[dataframe.index > backimg_df.index[0]]
#     print('sfsfieieieieie')


In [64]:
dataframe[dataframe['Leftsidemax'] == dataframe['Leftsidemax'].max()]['image_name'].values[0]

'snapshots37/9999snapshot_38.jpg'

In [65]:
dataframe[dataframe['Leftsidemax'] == dataframe['Leftsidemax'].max()]['image_name'].values[0]

'snapshots37/9999snapshot_38.jpg'

In [66]:
# 'snapshots38/9999snapshot_41.jpg', 'snapshots38/9999snapshot_105.jpg', snapshots37/9999snapshot_66.jpg

In [67]:
dataframe[dataframe['Rightsidemax'] == dataframe['Rightsidemax'].max()]['image_name'].values[0]

'snapshots37/9999snapshot_92.jpg'

In [224]:
secondside[secondside['sidemax'] == secondside['sidemax'].max()]['image_name'].values[0]

'snapshots38/9999snapshot_105.jpg'

In [214]:
firstside[firstside['sidemax'] == firstside['sidemax'].max()]['image_name']

66    snapshots37/9999snapshot_66.jpg
Name: image_name, dtype: object

In [197]:
fender_cordinates = firstside[firstside['sidemax'] == firstside['sidemax'].max()]['pred_cordinates'].values[0]['coordinates_FrontFender']
if (fender_cordinates[1]+fender_cordinates[3])/2 > 540:
    print('right')
else:
    print('left')
print(fender_cordinates)

left
[891, 386, 848, 480]


In [195]:
(fender_cordinates[1]+fender_cordinates[3]) /2

433.0

In [194]:
fender_cordinates

[891, 386, 848, 480]

In [198]:
fender_cordinates = secondside[secondside['sidemax'] == secondside['sidemax'].max()]['pred_cordinates'].values[0]['coordinates_FrontFender']
if (fender_cordinates[1]+fender_cordinates[3])/2 > 540:
    print('right')
else:
    print('left')
print(fender_cordinates)

left
[0, 449, 193, 480]


In [199]:
secondside[secondside['sidemax'] == secondside['sidemax'].max()]

,FrontBumper,FrontFender,FrontLid,FrontLight,FrontWindShield,Sidemirror,Top,Wheels,pred_cordinates,image_name,...,Frontsills,BackDoor,RearSideBumper,Rearsills,Sidesills,RearBumper,TailWindShield,FrontMax,BackMax,sidemax
91,1378.0,5722.0,3464.0,84.0,219.0,3353.0,37430,100408.0,"{'coordinates_BackDoor': [244, 352, 731, 480],...",snapshots37/9999snapshot_91.jpg,...,5016.0,194666.0,79452.0,19512.0,12620.0,96.0,61552.0,5145.0,108218.0,216770.0


In [201]:
fender_cordinates

[759, 307, 1250, 910]

In [202]:
1080/2

540.0

In [238]:
import requests

In [242]:
import requests

x = requests.get('https://lrnmxuci92.execute-api.ap-south-1.amazonaws.com/Prod/scratch-detection?Filepath=s3://vdi-data-prod1/26390405-f000-45e5-9d35-ab807329b8c4.jpg')

In [71]:
"""sdfsdfd """+ """sdlfjsdlfjsdlfkjsdf"""

'sdfsdfd sdlfjsdlfjsdlfkjsdf'

In [251]:
dict = {'name': 'sdfsdff', 'degree': 'sddfffffffffff', 'score': {'sdfsdff':22}} 
   
pd.DataFrame(dict)

,name,degree,score
sdfsdff,sdfsdff,sddfffffffffff,22
